**GPU SANITY CHECK**

In [1]:
import torch, os
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("CWD:", os.getcwd())


CUDA available: True
GPU: Tesla T4
CWD: /kaggle/working


**Cloning GitHub repository**
This repository contains all the code, which was tested on a local machine. Part of this code will be modified slightly later on to align to Kaggle's environment. 

In [2]:
!cd /kaggle/working
!git clone https://github.com/jonny0349/kaggle-sci-image-forgery-seg
!ls -la kaggle-sci-image-forgery-seg

Cloning into 'kaggle-sci-image-forgery-seg'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 74 (delta 29), reused 57 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 3.50 MiB | 20.23 MiB/s, done.
Resolving deltas: 100% (29/29), done.
total 40
drwxr-xr-x 6 root root 4096 Dec 18 16:33 .
drwxr-xr-x 3 root root 4096 Dec 18 16:33 ..
drwxr-xr-x 2 root root 4096 Dec 18 16:33 configs
drwxr-xr-x 8 root root 4096 Dec 18 16:33 .git
-rw-r--r-- 1 root root  242 Dec 18 16:33 .gitignore
drwxr-xr-x 2 root root 4096 Dec 18 16:33 notebooks
-rw-r--r-- 1 root root 5480 Dec 18 16:33 README.md
-rw-r--r-- 1 root root  368 Dec 18 16:33 requirements.txt
drwxr-xr-x 2 root root 4096 Dec 18 16:33 src


**Installation of dependencies**

We need some dependencies to be able to run the code in here, however, there was a conflict before with requirements.txt so we made sure to avoid installing TensorBoard/TensorFlow to avoid issues. We will install only what we need. 

In [3]:
!pip install -U --no-cache-dir \
    segmentation-models-pytorch \
    albumentations \
    timm \
    ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 354.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 414.0 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.20
    Uninstalling timm-1.0.20:
      Successfully uninstalled timm-1.0.20


**Patching train.py to disable TensorBoard safely**

This is done to avoid issues with TensorBoard/TensorFlow

In [4]:
from pathlib import Path
import re

p = Path("/kaggle/working/kaggle-sci-image-forgery-seg/src/train.py")
s= p.read_text()

# 1. Remove/guard SummaryWriter import
s = re.sub(r"from torch\.utils\.tensorboard import SummaryWriter.*\n", "SummaryWriter = None # disabled on Kaggle\n", s)

# 2. Replace writer initialization with writer=None
s = re.sub(r"writer\s*=\s*SummaryWriter\([^\)]*\)\s*", "writer = None\n", s)

# 3. Guard writer.close()
s = s.replace("writer.close()", "if writer is not None:\n  writer.close()")

p.write_text(s)
print("Disabled TensorBoard writer in train.py")

Disabled TensorBoard writer in train.py


**Patching data.py**

This is done so the algorithm can layout nested images in folders such as `train_images/authentic/...` since it wasn't happening before due to paths being different in Kaggle than in local environment. 

In [5]:
from pathlib import Path

p = Path("/kaggle/working/kaggle-sci-image-forgery-seg/src/data.py")
lines = p.read_text().splitlines()

# Find start/end or _collect_ids
start = None
for i, line in enumerate(lines):
    if line.lstrip().startswith("def _collect_ids("):
        start = i
        break
if start is None:
    raise RuntimeError("Could not find def _collect_ids")

base_indent = len(lines[start]) - len(lines[start].lstrip())
end = None
for j in range(start + 1, len(lines)):
    l = lines[j]
    if not l.strip():
        continue
    indent_len = len(l) - len(l.lstrip())
    if indent_len <= base_indent and (l.lstrip().startswith("def ") or l.lstrip().startswith("class ")):
        end = j
        break
if end is None:
    end = len(lines)

indent = " " * base_indent

new_fn = [
f"{indent}def _collect_ids(self) -> list[str]:",
f"{indent}    \"\"\"Collect matching (image, mask) pairs for Kaggle layout.",
f"{indent}",
f"{indent}    Images are nested (authentic/forged). Masks are flat .npy files.",
f"{indent}    \"\"\"",
f"{indent}    import os",
f"{indent}    from glob import glob",
f"{indent}",
f"{indent}    img_dir = self.images_dir",
f"{indent}    msk_dir = self.masks_dir",
f"{indent}    if not os.path.isdir(img_dir):",
f"{indent}        raise RuntimeError(f\"Images directory not found: {{img_dir}}\")",
f"{indent}    if not os.path.isdir(msk_dir):",
f"{indent}        raise RuntimeError(f\"Masks directory not found: {{msk_dir}}\")",
f"{indent}",
f"{indent}    exts = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp')",
f"{indent}    img_paths = []",
f"{indent}    for ext in exts:",
f"{indent}        img_paths.extend(glob(os.path.join(img_dir, '**', f'*{{ext}}'), recursive=True))",
f"{indent}",
f"{indent}    stem_to_img = {{}}",
f"{indent}    for ip in sorted(img_paths):",
f"{indent}        stem = os.path.splitext(os.path.basename(ip))[0]",
f"{indent}        stem_to_img.setdefault(stem, ip)",
f"{indent}",
f"{indent}    mask_paths = glob(os.path.join(msk_dir, '*.npy'))",
f"{indent}    stem_to_msk = {{os.path.splitext(os.path.basename(mp))[0]: mp for mp in mask_paths}}",
f"{indent}",
f"{indent}    ids = sorted(set(stem_to_img.keys()) & set(stem_to_msk.keys()))",
f"{indent}    if len(ids) == 0:",
f"{indent}        raise RuntimeError(",
f"{indent}            f\"No (image, mask) pairs found under {{img_dir}} and {{msk_dir}}.\\n\"",
f"{indent}            f\"Found images: {{len(stem_to_img)}} (sample={{list(stem_to_img)[:10]}})\\n\"",
f"{indent}            f\"Found masks : {{len(stem_to_msk)}} (sample={{list(stem_to_msk)[:10]}})\\n\"",
f"{indent}            f\"Expected matching stems between images and masks.\"",
f"{indent}        )",
f"{indent}",
f"{indent}    self._stem_to_img = stem_to_img",
f"{indent}    self._stem_to_msk = stem_to_msk",
f"{indent}    return ids"
]

lines = lines[:start] + new_fn + lines[end:]
p.write_text("\n".join(lines) + "\n")
print("Patched _collect_ids() for nested images + flat masks")

Patched _collect_ids() for nested images + flat masks


Now we need to correct img_path right before cv2.imread to prevent overwrites. 

In [6]:
from pathlib import Path

p = Path("/kaggle/working/kaggle-sci-image-forgery-seg/src/data.py")
lines = p.read_text().splitlines()

target_idx = None
for i, line in enumerate(lines):
    if "cv2.imread" in line:
        target_idx = i
        break
if target_idx is None:
    raise RuntimeError("Could not find cv2.imread in data.py")

indent = " " * (len(lines[target_idx]) - len(lines[target_idx].lstrip()))
inject = [
    f"{indent}# --- Kaggle nested layout support (force correct path right before read) ---",
    f"{indent}stem = self.ids[idx]",
    f"{indent}img_path = getattr(self, '_stem_to_img', {{}}).get(stem)",
    f"{indent}if img_path is None:",
    f"{indent}    from glob import glob",
    f"{indent}    cands = glob(os.path.join(self.images_dir, '**', f'{{stem}}.*'), recursive=True)",
    f"{indent}    img_path = cands[0] if cands else os.path.join(self.images_dir, f'{{stem}}.png')",
]

lines = lines[:target_idx] + inject + lines[target_idx:]
p.write_text("\n".join(lines) + "\n")
print("Injected forced img_path resolution before cv2.imread")

Injected forced img_path resolution before cv2.imread


** Creating configs**

We need to modify the configs/baseline.yaml to make sure we have the correct paths for Kaggle. First, we need to make a copy of the baseline.yaml and name it kaggle.yaml, then we will make the edits to adjust to the correct paths. 

In [7]:
from pathlib import Path
import yaml

base = Path("/kaggle/working/kaggle-sci-image-forgery-seg")
src_cfg = base / "configs" / "baseline.yaml"
dst_cfg = base / "configs" / "kaggle.yaml"

cfg = yaml.safe_load(src_cfg.read_text())

DATA_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
OUT_ROOT = str(base / "outputs")

cfg.setdefault("data", {})
cfg["data"]["root"] = DATA_ROOT
cfg["outputs"] = cfg.get("outputs", {})
cfg["outputs"]["root"] = OUT_ROOT

# Important: point to Kaggle's actual folder names
cfg["data"]["train_images_dir"] = f"{DATA_ROOT}/train_images"
cfg["data"]["train_masks_dir"] = f"{DATA_ROOT}/train_masks"
cfg["data"]["val_images_dir"] = f"{DATA_ROOT}/train_images" #Temporary until we make a true split
cfg["data"]["val_masks_dir"] = f"{DATA_ROOT}/train_masks"

# GPU settings
cfg.setdefault("project", {})
cfg["project"]["device"] = "cuda"

dst_cfg.write_text(yaml.safe_dump(cfg, sort_keys=False))
print("Wrote", dst_cfg)
print(dst_cfg.read_text().splitlines()[:30])

Wrote /kaggle/working/kaggle-sci-image-forgery-seg/configs/kaggle.yaml
['project:', '  seed: 2025', '  device: cuda', '  output_dir: outputs', 'data:', '  root: /kaggle/input/recodai-luc-scientific-image-forgery-detection', '  train_images_dir: /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images', '  train_masks_dir: /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks', '  val_images_dir: /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images', '  val_masks_dir: /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks', '  img_ext: .png', '  mask_ext: .npy', "  mask_suffix: ''", '  num_classes: 1', '  background_is_zero: true', 'augment:', '  train:', '    resize:', '    - 768', '    - 768', '    hflip_p: 0.5', '    vflip_p: 0.0', '    rotate_limit: 10', '    rotate_p: 0.25', '    brightness_contrast_p: 0.15', '  val:', '    resize:', '    - 768', '    - 768', 'model:']


**Running Training**

Now we are ready to train the model

In [8]:
!cd /kaggle/working/kaggle-sci-image-forgery-seg && python -m src.train --cfg configs/kaggle.yaml

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 159, in _get_module_details
  File "<frozen importlib._bootstrap_external>", line 1133, in get_code
  File "<frozen importlib._bootstrap_external>", line 1063, in source_to_code
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/kaggle/working/kaggle-sci-image-forgery-seg/src/train.py", line 126
    save_json(cfg.to_dict(), os.path.join(out_dir, "config_snapshot.json"))
IndentationError: unexpected indent


**Run Inference**

We run inference just for sanity check

In [9]:
!cd /kaggle/working/kaggle-sci-image-forgery-seg && python -m src.infer \
  --cfg configs/kaggle.yaml \
  --checkpoint /kaggle/working/kaggle-sci-image-forgery-seg/outputs/checkpoints/best_dice.pt \
  --input_dir /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged \
  --output_dir /kaggle/working/outputs/preds/forged_demo \
  --thr 0.5 \
  --save_overlay

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [10]:
!cp /kaggle/working/kaggle-sci-image-forgery-seg/outputs/checkpoints/best_dice.pt /kaggle/working/best_dice.pt

cp: cannot stat '/kaggle/working/kaggle-sci-image-forgery-seg/outputs/checkpoints/best_dice.pt': No such file or directory


In [11]:
!cd /kaggle/working/kaggle-sci-image-forgery-seg && \
    zip -r /kaggle/working/artifacts.zip outputs/checkpoints outputs/logs outputs/preds outputs/config_snapshot.json

	zip warning: name not matched: outputs/checkpoints
	zip warning: name not matched: outputs/logs
	zip warning: name not matched: outputs/preds
	zip warning: name not matched: outputs/config_snapshot.json

zip error: Nothing to do! (try: zip -r /kaggle/working/artifacts.zip . -i outputs/checkpoints outputs/logs outputs/preds outputs/config_snapshot.json)
